# 1.01 County Data Cleanup

In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
import lib.county_data as ctd
import lib.name_categories as nc
import re
from lib.collapse_facil_dict import event_facil_addresses
from sklearn.externals import joblib

## I. Load & Clean Data

Download data from LA Public Health website (https://ehservices.publichealth.lacounty.gov/ezsearch) and read the data in using pandas

In [3]:
rest_df = pd.read_csv('data/LA_rest_inspects', delimiter = '\t', header = 0, index_col = False,
                             names = ['facility', 'last_inspect_date','inspect_score', 'descript',
                                      'address', 'city', 'zip', 'phone'])

In [4]:
rest_df.head()

,facility,last_inspect_date,inspect_score,descript,address,city,zip,phone
0,103RD MARKET,01/18/2017,95.0,"FOOD MKT RETAIL (25-1,999 SF) LOW RISK",1922 E 103RD ST,LOS ANGELES,90002,323-563-6245
1,TACOS LA POTRANKA,01/18/2017,85.0,RESTAURANT (0-30) SEATS HIGH RISK,1501 N LONG BEACH BLVD,COMPTON,90221-1608,323-928-1229
2,FISHLAND,01/18/2017,97.0,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",510 W COMPTON BLVD,COMPTON,90220,310-669-3050
3,PRODUCE FOR LESS,01/18/2017,91.0,"FOOD MKT RETAIL (2,000+ SF) HIGH RISK",5059 MELROSE AVE,LOS ANGELES,90038-4115,323-466-2966
4,TREE HOUSE THAI RESTAURANT,01/18/2017,92.0,RESTAURANT (31-60) SEATS HIGH RISK,10200 VENICE BLVD STE 207,CULVER CITY,90232,NaN


## II. Clean the Data

Note, all functions are stored in a separate `county_data.py` file.

- strip columns of any spaces when possible
- filter out any food facilities that do not serve customers (i.e. retail)
- extract necessary data and move into separate columns (seat range, risk, zip codes, phone numbers)
- remove key words that may not help the Yelp search

In [5]:
ctd.strip_col_contents(rest_df)

SUCCESS     : facility column stripped
SUCCESS     : last_inspect_date column stripped
UNSUCCESSFUL: inspect_score column not stripped due to format
SUCCESS     : descript column stripped
SUCCESS     : address column stripped
SUCCESS     : city column stripped
SUCCESS     : zip column stripped
UNSUCCESSFUL: phone column not stripped due to format


In [7]:
rest_df = ctd.filter_dataframe('restaurant', 'descript', rest_df)

In [10]:
rest_df[rest_df['city'] == 'LOS ANGELES'].sort_values('facility')

,facility,last_inspect_date,inspect_score,descript,address,city,zip,phone
22794,# 245 FRIES FRIES FRIES,09/21/2016,96.0,RESTAURANT (0-30) SEATS HIGH RISK,1000 VIN SCULLY AVE,LOS ANGELES,90012,NaN
28835,# 431 LOGE TERRACE BAR,08/09/2016,99.0,RESTAURANT (0-30) SEATS MODERATE RISK,1000 VIN SCULLY AVE,LOS ANGELES,90012,NaN
22608,# 953 TOP DECK DOGS,09/21/2016,98.0,RESTAURANT (151 + ) SEATS HIGH RISK,1000 VIN SCULLY AVE,LOS ANGELES,90012,NaN
27476,#1 CAFE,08/18/2016,90.0,RESTAURANT (0-30) SEATS HIGH RISK,2080 CENTURY PARK E STE 108,LOS ANGELES,90067,NaN
28626,#1 CHINESE FAST FOOD,08/10/2016,92.0,RESTAURANT (0-30) SEATS HIGH RISK,8606 S VERMONT AVE,LOS ANGELES,90044,213-235-8779
34914,#1 DONUT,05/17/2016,98.0,RESTAURANT (0-30) SEATS MODERATE RISK,8509 S FIGUEROA ST,LOS ANGELES,90003,NaN
30533,#1 DONUTS,07/26/2016,91.0,RESTAURANT (0-30) SEATS MODERATE RISK,8509 S FIGUEROA ST # 106,LOS ANGELES,90003,626-695-7898
29068,#102 ALL INCLUSIVE (AYCE),08/08/2016,98.0,RESTAURANT (0-30) SEATS MODERATE RISK,1000 VIN SCULLY AVE,LOS ANGELES,90012,NaN
29073,#103 DODGER DOG TRADITIONAL,08/08/2016,99.0,RESTAURANT (0-30) SEATS MODERATE RISK,1000 VIN SCULLY AVE,LOS ANGELES,90012,NaN
29119,#104 ALL INCLUSIVE (AYCE),08/08/2016,99.0,RESTAURANT (0-30) SEATS MODERATE RISK,1000 VIN SCULLY AVE,LOS ANGELES,90012,NaN


In [7]:
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub(' STE$','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub(' STE .+','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub(' STE#.+','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub(' # .+','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub(' #.+','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub('#\d+','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub('# \d+','',x))
rest_df['address'] = rest_df['address'].apply(lambda x: re.sub('#$','',x))

In [8]:
rest_df['max_seats'] = rest_df['descript'].str.extract('RESTAURANT \((.+)\)', expand = True)

In [9]:
rest_df['max_seats'] = rest_df['max_seats'].apply(ctd.max_seating)

In [10]:
rest_df['phone'] = rest_df['phone'].str.replace('-','')

In [11]:
rest_df['risk_level'] = rest_df['descript'].str.extract('\) SEATS (.+) RISK', expand = True)

In [12]:
rest_df['risk_level'] = rest_df['risk_level'].apply(ctd.risk_lev)

In [13]:
rest_df['zip_cd'] = rest_df['zip'].str.extract('(\d{5})', expand = True)

In [14]:
rest_df['facility'] = rest_df['facility'].str.replace(',', '')
rest_df['facility'] = rest_df['facility'].str.replace("'", '')
rest_df['facility'] = rest_df['facility'].str.replace(' #\d+', '')
rest_df['facility'] = rest_df['facility'].str.replace(' # \d+', '')
rest_df['facility'] = rest_df['facility'].str.replace(' INC', '')

In [15]:
rest_df.drop('descript', axis = 1, inplace = True)
rest_df.drop('zip', axis = 1, inplace = True)
rest_df.dropna(axis=0, subset=['zip_cd'], inplace=True)

In [16]:
rest_df['zip_cd'] = rest_df['zip_cd'].astype('str')

In [17]:
rest_df['phone'] = rest_df['phone'].apply(lambda x: '+1' + str(x))

In [18]:
rest_df = rest_df.groupby(['facility', 'address', 'city', 'zip_cd'], 
                            as_index = False).agg({'max_seats': 'max',
                                                   'risk_level': 'max',
                                                   'last_inspect_date': 'max',
                                                   'inspect_score': 'mean',
                                                   'phone': 'min'})
rest_df.head()

,facility,address,city,zip_cd,max_seats,phone,risk_level,last_inspect_date,inspect_score
0,# 245 FRIES FRIES FRIES,1000 VIN SCULLY AVE,LOS ANGELES,90012,30,+1nan,3,09/21/2016,96.0
1,# 431 LOGE TERRACE BAR,1000 VIN SCULLY AVE,LOS ANGELES,90012,30,+1nan,2,08/09/2016,99.0
2,# 953 TOP DECK DOGS,1000 VIN SCULLY AVE,LOS ANGELES,90012,151,+1nan,3,09/21/2016,98.0
3,#1 BUFFET,14418 HAWTHORNE BLVD,LAWNDALE,90260,151,+13106444007,3,11/29/2016,90.0
4,#1 CAFE,2080 CENTURY PARK E,LOS ANGELES,90067,30,+1nan,3,08/18/2016,90.0


In [19]:
rest_cities = rest_df.groupby('city', as_index = False).agg({'facility': 'count'})

In [20]:
rest_cities = rest_cities.sort_values('facility', ascending = False)

### A. Narrow the Scope

- focus on restaurants in neighbrohoods with over 100 restaurants, so we can take saturation into account later
- note that Los Angeles is quite broad, and contains 28% of the entire data set


In [21]:
rest_cities.head()

,city,facility
81,LOS ANGELES,9945
152,TORRANCE,810
132,SANTA MONICA,782
46,GLENDALE,719
20,BURBANK,584


In [22]:
rest_cities['filter'] = rest_cities['facility'].apply(lambda x: 1 if x > 100 else 0)

In [23]:
rest_cities = ctd.filter_and_drop_column(rest_cities)

In [24]:
rest_scope = rest_cities['city'].values

In [25]:
rest_df['filter'] = rest_df['city'].apply(lambda x: 1 if x in rest_scope else 0)

In [26]:
rest_df = ctd.filter_and_drop_column(rest_df)

### B. Consolidate the Data

There's a lot of potentially duplicative data. We need to differentiate between a couple of things:
- multiple restaurant offerings at one address
- restaurant turnover at the same address
- the same restaurant receiving yearly inspections under different names

To better understand and solve this problem, I'm going to create a dictionary of addresses, with the values as a list of different restaurant names.

In addition, we have another check coming on the Yelp API, so we don't have to spend too much time on this.

In [27]:
rest_facil_dict = rest_df.groupby('address').groups

In [28]:
for key, value in rest_facil_dict.items():
    index_list = []
    for index_num in value:
        index_num = rest_df.ix[index_num]['facility']
        index_list.append(index_num)
    rest_facil_dict[key] = index_list

In [29]:
def num_rest(x):
    if x in rest_facil_dict:
        return len(rest_facil_dict[x])

rest_df['num_at_address'] = rest_df['address'].apply(num_rest)

In [30]:
rest_df['facility'] = rest_df.apply(lambda col: ctd.rename_facilities(col['address'],
                                                                      col['facility']), axis = 1)

In [31]:
rest_group_df = rest_df.groupby(['facility', 'address', 'city', 'zip_cd', 'num_at_address'], 
                                as_index = False).agg({'max_seats': 'max',
                                                       'risk_level': 'max',
                                                       'last_inspect_date': 'max',
                                                       'inspect_score': 'mean',
                                                       'phone': 'min'})

In [32]:
rest_group_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31996 entries, 0 to 31995
Data columns (total 10 columns):
facility             31996 non-null object
address              31996 non-null object
city                 31996 non-null object
zip_cd               31996 non-null object
num_at_address       31996 non-null int64
max_seats            31996 non-null int64
phone                31996 non-null object
risk_level           31996 non-null int64
last_inspect_date    31996 non-null object
inspect_score        31996 non-null float64
dtypes: float64(1), int64(3), object(6)
memory usage: 2.7+ MB


In [33]:
rest_group_df['cat'] = rest_group_df['facility'].apply(nc.define_cat)

In [34]:
rest_group_df = rest_group_df[rest_group_df['cat'].isnull()]

In [35]:
rest_group_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30850 entries, 0 to 31995
Data columns (total 11 columns):
facility             30850 non-null object
address              30850 non-null object
city                 30850 non-null object
zip_cd               30850 non-null object
num_at_address       30850 non-null int64
max_seats            30850 non-null int64
phone                30850 non-null object
risk_level           30850 non-null int64
last_inspect_date    30850 non-null object
inspect_score        30850 non-null float64
cat                  0 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 2.8+ MB


In [36]:
la_rest_group_df = rest_group_df[rest_group_df['city'] == 'LOS ANGELES']

In [37]:
pd.to_pickle(la_rest_group_df, 'data/la_rest_group_df.pkl')